# Linear Programming Optimization using Simplex Method

In this notebook, we will solve a linear programming (LP) problem using the Simplex method. We will define the objective function and constraints, convert the problem to its standard form, and iteratively perform pivot operations to find the optimal solution. The steps include:

1. Defining the LP model.
2. Converting the model to standard form.
3. Creating the initial simplex tableau.
4. Performing pivot operations to optimize the objective function.
5. Interpreting the final tableau to obtain the optimal solution.


## Probleme Definition

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import symbols, Eq, solve
from IPython.display import display, Math

# Define the symbols
t, c = sp.symbols('t c')

# Define the LP model
opdir = 'max'
objective = 500*t + 300*c
constraints = [2*t + c <= 40, t + 2*c <= 50, t >= 0, c >= 0]

# Print the LP model
display(Math(opdir + ' \\; Z = ' + sp.latex(objective)))
display(Math('\\text{s.t. }' + sp.latex(constraints)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Standard Form

In [9]:
# Define slack variables
s1, s2 = sp.symbols('s1 s2')

# Convert inequalities to equalities by adding slack variables
standard_constraints = [
    Eq(2*t + c + s1, 40),
    Eq(t + 2*c + s2, 50),
    t>=0,
    c>=0,
    s1>=0,
    s2>=0
]

# Print the standard form of the LP model
display(Math(opdir + ' \\; Z = ' + sp.latex(objective)))
display(Math('\\text{s.t. }' + sp.latex(standard_constraints)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Simplex (First Tableau)

In [10]:
import pandas as pd

# Create a function to generate the simplex tableau
def create_simplex_tableau(standard_constraints, objective):
    # Extract coefficients for the constraints
    tableau = []
    for constraint in standard_constraints[:-4]:  # Exclude non-negativity constraints
        coeffs = [constraint.lhs.coeff(var) for var in [t, c, s1, s2]]
        rhs = constraint.rhs
        tableau.append(coeffs + [rhs])
    
    # Add the objective function row
    obj_coeffs = [-objective.coeff(var) for var in [t, c, s1, s2]]
    tableau.append(obj_coeffs + [0])
    
    # Create a pandas DataFrame
    columns = ['BV', 't', 'c', 's1', 's2', 'rhs']
    index = ['s1', 's2', 'Z'][:len(tableau)]
    df = pd.DataFrame(tableau, columns=columns[1:], index=index)
    #df.insert(0, 'BV', index)
    
    
    return df

# Generate the simplex tableau
simplex_tableau = create_simplex_tableau(standard_constraints, objective)

# Display the simplex tableau
display(simplex_tableau)





,t,c,s1,s2,rhs
s1,2,1,1,0,40
s2,1,2,0,1,50
Z,-500,-300,0,0,0


## Second Tableau

In [11]:
import numpy as np

def pivot_on(tableau, row, col):
    # Convert the data types to numeric values
    #tableau = tableau.apply(pd.to_numeric, errors='coerce')
    
    # Divide the pivot row by the pivot element
    tableau.iloc[row] = tableau.iloc[row] / tableau.iloc[row, col]
    # Subtract a multiple of the pivot row from all other rows
    for i in range(len(tableau)):
        if i == row:
            continue
        tableau.iloc[i] = tableau.iloc[i] - tableau.iloc[i, col] * tableau.iloc[row]
    
    # Update the index of the pivot row
    tableau.index.values[row] = tableau.columns[col]
    
    return tableau

# Define the pivot column
pivot_col = 't'

# Find the pivot row
pivot_row = np.argmin(simplex_tableau.iloc[:-1]['rhs'] / simplex_tableau.iloc[:-1][pivot_col])

display('pivot column:'+str(pivot_col))
display('pivot row:'+str(pivot_row))
# Perform the pivot operation
pivot_on(simplex_tableau, pivot_row, simplex_tableau.columns.get_loc(pivot_col))

'pivot column:t'

'pivot row:0'

,t,c,s1,s2,rhs
t,1,1/2,1/2,0,20
s2,0,3/2,-1/2,1,30
Z,0,-50,250,0,10000


## Third Tableau

In [12]:
pivot_col = 'c'
pivot_row = np.argmin(simplex_tableau.iloc[:-1]['rhs'] / simplex_tableau.iloc[:-1][pivot_col])
display('pivot column:'+str(pivot_col))
display('pivot row:'+str(pivot_row))
# Perform the pivot operation
pivot_on(simplex_tableau, pivot_row, simplex_tableau.columns.get_loc(pivot_col))

'pivot column:c'

'pivot row:1'

,t,c,s1,s2,rhs
t,1,0,2/3,-1/3,10
c,0,1,-1/3,2/3,20
Z,0,0,700/3,100/3,11000


## Conclusion

The optimal solution is found in the last row of the simplex tableau. The values of the decision variables can be read from the tableau where the basic variables (BV) are equal to the decision variables.

The optimal solution is:
- $ t = 10 $
- $ c = 20 $
- Objective value $ Z = 11000 $